In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import pandas as pd
import numpy as np
import re
import keras
from tqdm import tqdm
import pickle as pkl
from tensorflow.keras import layers
import tensorflow as tf
import gc
from sklearn.metrics import accuracy_score
import os
gc.enable()

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, TimeDistributed, Activation, Masking, Input
from tensorflow.keras.layers import LSTM, Bidirectional, dot, concatenate
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import * 
from tensorflow.keras.models import load_model
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.backend import squeeze
from tensorflow.keras. optimizers import RMSprop, Adam

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
keras.backend.set_session(sess)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17008425042718992300
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 2195864329879719364
physical_device_desc: "device: XLA_CPU device"
]
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device



## Seq2Seq по предыдущему

In [ ]:
order_features = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Diplom/instacart-market-basket-analysis/test_train/order_features.csv')
order_features['prod_seq'] = order_features['prod_seq'].apply(lambda x: list(np.fromstring(x.strip('[ ]'), dtype=int, sep=', ')))
order_features['prev_seq'] = order_features['prev_seq'].apply(lambda x: list(np.fromstring(x.strip('[ ]'), dtype=int, sep=', ')))
order_features['prev_orders_seq'] = order_features['prev_orders_seq'].apply(lambda x: list(np.fromstring(x.strip('[ ]'), dtype=int, sep=', ')))

train_positive = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Diplom/instacart-market-basket-analysis/test_train/train_positive.csv')
train_negative = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Diplom/instacart-market-basket-analysis/test_train/train_negative.csv')

with open('/content/drive/My Drive/Colab Notebooks/Diplom/instacart-market-basket-analysis/test_train/id_to_token.pkl', 'rb') as f:
  id_to_token = pkl.load(f)

with open('/content/drive/My Drive/Colab Notebooks/Diplom/instacart-market-basket-analysis/test_train/token_to_id.pkl', 'rb') as f:
  token_to_id = pkl.load(f)

with open('/content/drive/My Drive/Colab Notebooks/Diplom/instacart-market-basket-analysis/test_train/val_orders.pkl', 'rb') as f:
  val_orders = pkl.load(f)

with open('/content/drive/My Drive/Colab Notebooks/Diplom/instacart-market-basket-analysis/test_train/test_orders.pkl', 'rb') as f:
  test_orders = pkl.load(f)

In [ ]:
it = 2
os.makedirs('/content/drive/My Drive/Colab Notebooks/Diplom/models/seq2seq/{0}'.format(it))

In [ ]:
df_train = order_features[(~order_features['order_id'].isin(val_orders)) & (~order_features['order_id'].isin(test_orders))].reset_index(drop=True)
df_val = order_features[order_features['order_id'].isin(val_orders)].reset_index(drop=True)
df_test = order_features[order_features['order_id'].isin(test_orders)].reset_index(drop=True)

###Создаем модель

In [ ]:
for el in ['<START>', '<END>']:
  id_to_token[len(id_to_token)] = el

for i_d, token in id_to_token.items():
  if token not in token_to_id:
    token_to_id[token] = i_d

In [ ]:
vocab_size = len(id_to_token)
hidden_size = 10

encoder_inputs = Input(shape=(None,), name='input_encoder')
decoder_inputs = Input(shape=(None,), name='input_decoder')

embeddings = Embedding(vocab_size + 1, 13, mask_zero=True, name='embedding')

encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder_outputs, state_h, state_c = LSTM(hidden_size, return_sequences=True, return_state=True, name='lstm_encoder')(encoder_embeddings)

decoder_outputs, _, _ = LSTM(hidden_size, return_sequences=True, return_state=True, name='lstm_decoder')(decoder_embeddings, initial_state=[state_h, state_c])

attention = dot([decoder_outputs, encoder_outputs], axes=[2, 2], name='score_attention')
attention = Activation('softmax', name='attention_activation')(attention)

context = dot([attention, encoder_outputs], axes=[2,1], name='attention_weighted_dot')
decoder_combined_context = concatenate([context, decoder_outputs], name='context')

output = TimeDistributed(Dense(hidden_size, activation='tanh'), name='dense_tanh')(decoder_combined_context)
output = TimeDistributed(Dense(vocab_size+1, activation='softmax'), name='dense')(output)

model = Model([encoder_inputs, decoder_inputs], output)

optimizer = Adam(learning_rate=0.00001)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_decoder (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
input_encoder (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 13)     322673      input_encoder[0][0]              
                                                                 input_decoder[0][0]              
__________________________________________________________________________________________________
lstm_encoder (LSTM)             [(None, None, 10), ( 960         embedding[0][0]            

###Делаем батч генератор

In [ ]:
def batch_generator(data, batch_size, vocab_token):
  step = 0
  while True:
    if step * batch_size > len(data):
      step = 0

    tr_temp = data[step * batch_size:(step+1) * batch_size]

    maxlen_encoder = max(list(map(lambda x: len(x[0]), tr_temp)))
    maxlen_decoder = max(list(map(lambda x: len(x[1]), tr_temp))) + 2

    x_encoder = np.zeros((tr_temp.shape[0], maxlen_encoder), dtype=int)
    x_decoder = np.zeros((tr_temp.shape[0], maxlen_decoder), dtype=int)
    y_decoder = np.zeros((tr_temp.shape[0], maxlen_decoder, 1), dtype=int)

    for i in range(len(tr_temp)):
      source = tr_temp[i][0]
      target = ['<START>'] + tr_temp[i][1] + ['<END>']
      for j in range(len(source)):
        x_encoder[i,j] = vocab_token[source[j]] + 1
      for k in range(len(target)-1):
        x_decoder[i,k] = vocab_token[target[k]] + 1
        y_decoder[i,k,0] = vocab_token[target[k+1]] + 1

    step += 1

    yield [x_encoder, x_decoder], y_decoder

In [ ]:
for x, y in batch_generator(df_train[df_train['order_number'] != 1][['prod_seq', 'prev_seq']].values, 64, token_to_id):
  break

###Делаем функцию для семплинга последовательностей

In [ ]:
encoder_sampling_input = Input(shape=(None,))
encoder_sampling_embedding = model.get_layer(name='embedding')(encoder_sampling_input)
encoder_sampling_output, encoder_sampling_state_h, encoder_sampling_state_c = model.get_layer(name='lstm_encoder')(encoder_sampling_embedding)
encoder_states = [encoder_sampling_state_h, encoder_sampling_state_c]
encoder_sampling_model = Model(encoder_sampling_input, encoder_states)

decoder_sampling_input = Input(shape=(None,))
decoder_sampling_h = Input(shape=(hidden_size,))
decoder_sampling_c = Input(shape=(hidden_size,))
decoder_sampling_states_init = [decoder_sampling_h, decoder_sampling_c]
decoder_sampling_embedding = model.get_layer(name='embedding')(decoder_sampling_input)
decoder_sampling_output, decoder_sampling_state_h, decoder_sampling_state_c = model.get_layer(name='lstm_decoder')(decoder_sampling_embedding, initial_state=decoder_sampling_states_init)
decoder_sampling_output_states = [decoder_sampling_state_h, decoder_sampling_state_c]
output = model.get_layer(name='dense')(decoder_sampling_output)
decoder_sampling_model = Model([decoder_sampling_input] + decoder_sampling_states_init, [output] + decoder_sampling_output_states)

In [ ]:
def sampling(input_sequence, max_output_seq_length):
  
  input_sequence = np.reshape(list(map(lambda x: token_to_id[x]+1, input_sequence)), (1, -1))

  states = encoder_sampling_model.predict(input_sequence)

  decoder_input_token = np.zeros((1, 1), dtype='int')
  decoder_input_token[0,0] = token_to_id['<START>'] + 1

  target_seq = []

  isEnd = False

  while not isEnd:
    output_probs, state_h, state_c = decoder_sampling_model.predict([decoder_input_token] + states)

    states = [state_h, state_c]
    token_index = np.argmax(output_probs)
    decoder_input_token[0,0] = token_index
    target_seq.append(token_index-1)

    if id_to_token[token_index-1] == '<END>':
      isEnd = True

    if len(target_seq) >= max_output_seq_length:
      isEnd = True

  return list(map(lambda x: id_to_token[x], target_seq))

###Обучаем

In [ ]:
path = '/content/drive/My Drive/Colab Notebooks/Diplom/models/seq2seq/{0}'.format(it)

In [ ]:
def my_learning_rate(epoch, lrate):
  return lrate

checkpoint = ModelCheckpoint(path + 'seq2seq_epochs:{epoch:03d}_loss:{loss:.3f}_accuracy:{sparse_categorical_accuracy:.3f}_\
valloss:{val_loss:.3f}_valaccuracy:{val_sparse_categorical_accuracy:.3f}.hdf5', 
                             monitor='val_loss', verbose=1, save_best_only=True, mode='auto', save_freq='epoch')
stop = EarlyStopping(monitor='loss', min_delta=0.0001)
lrs = LearningRateScheduler(my_learning_rate)

callbacks_list = [checkpoint]

batch_size = 64
epochs = 25

In [ ]:
model.load_weights(path+'seq2seq_epochs:025_loss:3.035_accuracy:0.095_valloss:2.562_valaccuracy:0.098.hdf5')

In [ ]:
model.fit_generator(batch_generator(df_train[df_train['order_number'] != 1][['prod_seq', 'prev_seq']].values, batch_size, token_to_id), 
                    steps_per_epoch=df_train.shape[0]//batch_size, epochs=epochs, 
                    validation_data=batch_generator(df_val[['prod_seq', 'prev_seq']].values, batch_size, token_to_id), 
                    validation_steps=df_val.shape[0]//batch_size,  verbose=1, callbacks=callbacks_list)

### Проверяем качество на валидации

In [ ]:
precision_lstm = {}
recall_lstm = {}
for i in range(1, 11):
  precision_lstm[i] = 0
  recall_lstm[i] = 0

total = 0

for order_id, prev_seq in df_val[['order_id', 'prev_seq']].values:

  for i in range(1, 11):
    prod_set_features = sampling(prev_seq, i)
    recommended_lstm = list(map(lambda x: id_to_token[x], prod_set_features))
    precision_lstm[i] += np.intersect1d(bought, recommended_lstm).shape[0]/len(recommended_lstm)
    recall_lstm[i] += np.intersect1d(bought, recommended_lstm).shape[0]/len(bought)

  total += 1

for i in range(1, 11):
  precision_lstm[i] = precision_lstm[i]/total
  recall_lstm[i] = recall_lstm[i]/total

In [ ]:
for i in range(1, 11):
  precision_lstm[i] = precision_lstm[i]/total
  recall_lstm[i] = recall_lstm[i]/total

In [ ]:
precision_lstm

{1: 0.06349206349206349,
 2: 0.03571428571428571,
 3: 0.03439153439153439,
 4: 0.025793650793650792,
 5: 0.020634920634920638,
 6: 0.017195767195767195,
 7: 0.014739229024943307,
 8: 0.012896825396825396,
 9: 0.011463844797178132,
 10: 0.010317460317460319}

In [ ]:
recall_lstm

{1: 0.0052431359017036765,
 2: 0.006376922749776239,
 3: 0.008619301182630865,
 4: 0.008619301182630865,
 5: 0.008619301182630865,
 6: 0.008619301182630865,
 7: 0.008619301182630865,
 8: 0.008619301182630865,
 9: 0.008619301182630865,
 10: 0.008619301182630865}